In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

# player_ids = ['677950', '572233', '682998', '666150', '656896', '553993', '672515', '672695', '607054', '664983', '645444', '606466', '666971', '656976', '592626']
timeout = 25
# mlb_seasons = ['2023', '2022', '2021']
player_ids = [['dbacks', 'Alek Thomas', 'alek-thomas-677950'], ['dbacks', 'Ryne Nelson', 'href="/player/694297", 'Pitcher']]


In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [ ]:
url = f"https://baseballsavant.mlb.com/savant-player/alek-thomas-677950?stats=statcast-r-hitting-mlb"
driver.get(url)

pitches = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartBreakdown"]'))
pitches.select_by_index(0)

handedness = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartHandedness"]'))
handedness.select_by_index(1)

headers_present = EC.visibility_of_element_located((By.XPATH, r'//*[@id="detailedPitches"]/tbody/tr[1]/td[2]/span'))
WebDriverWait(driver, timeout).until(headers_present)

src = driver.page_source
parser = BeautifulSoup(src, "lxml")
table = parser.find("div", attrs = {"id": "pitchTable"})

headers = table.findAll('th')
headerlist = [h.text.strip() for h in headers[:]]
headerlist.insert(0, "Name")
headerlist.insert(2, "Team")


In [ ]:
headerlist 

In [ ]:
for player in player_ids:
    url = f"https://baseballsavant.mlb.com/savant-player/{player[-1]}?stats=statcast-r-hitting-mlb"
    driver.get(url)

    pitches = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartBreakdown"]'))
    pitches.select_by_index(0)

    handedness = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartHandedness"]'))

    for hand in range(1, len(handedness.options)):
        handedness.select_by_index(hand)
        
        first_row_present = EC.visibility_of_element_located((By.XPATH, r'//*[@id="detailedPitches"]'))
        WebDriverWait(driver, timeout).until(first_row_present)

        src = driver.page_source
        parser = BeautifulSoup(src, "lxml")
        table = parser.find("div", attrs = {"id": "pitchTable"})
    
        rows = table.findAll('tr')[1:]
    
        player_stats = []
        for i in range(len(rows)):
            row_cells = []
            for td in rows[i].findAll('td'):
                row_cells.append(td.getText().strip())
            row_cells.insert(0, player[1])
            row_cells.insert(2, player[0])
            player_stats.append(row_cells)
    

In [ ]:
driver.quit()

In [ ]:
player_stats

In [ ]:
rows

In [ ]:
df = pd.DataFrame(player_stats, columns=headerlist)

In [ ]:
df

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


payload = {}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:122.0) Gecko/20100101 Firefox/122.0',
  'Accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.5',
  'Accept-Encoding': 'gzip, deflate, br',
  'X-Requested-With': 'XMLHttpRequest',
  'Connection': 'keep-alive',
  'Referer': 'https://baseballsavant.mlb.com/savant-player/alek-thomas-677950?stats=statcast-r-hitting-mlb',
  'Cookie': '__cflb=02DiuJFcgbX2zpcHdRurfpjkBTD5BPySo6gPFqPfkQ556; AMCV_A65F776A5245B01B0A490D44%40AdobeOrg=1099438348%7CMCIDTS%7C19755%7CMCMID%7C08223760795725670020171971273569729942%7CMCAAMLH-1707411436%7C7%7CMCAAMB-1707411436%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1706813836s%7CNONE%7CMCAID%7CNONE%7CMCSYNCSOP%7C411-19762%7CvVersion%7C2.1.0; s_cc=true; s_sq=%5B%5BB%5D%5D; AMCVS_A65F776A5245B01B0A490D44%40AdobeOrg=1; s_vi=[CS]v1|32DDE6B6150B13BF-40000A6BE1F97D36[CE]; s_ecid=MCMID%7C08223760795725670020171971273569729942; _dd_s=rum=0&expire=1706807948264',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'TE': 'trailers'
}


players = [['dbacks', 'Alek Thomas', '677950']]
handedness =['R', 'L']

In [ ]:
for player in players:
    player_stats = []
    for hand in handedness:
        url = f"https://baseballsavant.mlb.com/player-services/statcast-pitches-breakdown?playerId={player[-1]}&position=8&hand={hand}&pitchBreakdown=pitches&timeFrame=yearly&season=&pitchType=&count=&updatePitches=true"

        r = requests.get(url, headers=headers)

        parser = BeautifulSoup(r.content, "html.parser")
        table = parser.find(id="detailedPitches")

        headersTH = table.find_all('th')
        headerlist = [h.text.strip() for h in headersTH[:]]
        headerlist.insert(0, "Name")
        headerlist.insert(2, "Team")
        headerlist.insert(3, "Hand") 
    
        rows = table.findAll('tr')[1:]
    
        for i in range(len(rows)):
            row_cells = []
            for td in rows[i].findAll('td'):
                row_cells.append(td.getText().strip())
            row_cells.insert(0, player[1])
            row_cells.insert(2, player[0])
            row_cells.insert(3, hand)
            player_stats.append(row_cells)

In [ ]:
headerlist

In [ ]:
player_stats

In [ ]:
df = pd.DataFrame(player_stats, columns=headerlist)


In [ ]:
df

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

timeout = 25

handedness =['R', 'L']
player_stats = []
headerlist = []
pitcher_stats = []
players = [['dbacks', 'Miguel Castro', 'miguel-castro-612434', 'Pitcher']]
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [ ]:
def missing_stats(pitches_less, pitches_more):  
    pitches_needed = []
    for row in pitches_more:
        yearPitch = row.findAll('td')[0].getText().strip() + row.findAll('td')[1].getText().strip()
        pitches_needed.append(yearPitch)
    
    pitches_got = []
    for row in pitches_less:
        yearPitch = row.findAll('td')[0].getText().strip() + row.findAll('td')[1].getText().strip()
        pitches_got.append(yearPitch)

    missing_pitch = []
    for pitch in pitches_needed:
        if pitch not in pitches_got:
            missing_pitch.append(pitch)

    missing_pitch_arr = []
    missing_pitch_arr.append(missing_pitch[0][:4])
    missing_pitch_arr.append(missing_pitch[0][4:])

    # for stat in spinDirectionRows[2:]:
    #     missing_pitch_arr.append('') # appends empty string for value to keep the correct amount of cols
    
    # spinDirection_stats.append(missing_pitch_arr) # appends missing pitch

In [ ]:
for player in players:
    url = f"https://baseballsavant.mlb.com/savant-player/{player[2]}?stats=statcast-r-pitching-mlb"
    driver.get(url)

    detailedPitches_present = EC.visibility_of_element_located((By.XPATH, r'//*[@id="detailedPitches"]/tbody/tr[1]/td[2]/span'))
    WebDriverWait(driver, timeout).until(detailedPitches_present)

    src = driver.page_source
    parser = BeautifulSoup(src, "lxml")

    pitchMovement = parser.find("table", attrs = {"id": "pitchMovement"})
    pitchTracking = parser.find("table", attrs = {"id": "detailedPitches"})
    runValues = parser.find("table", attrs = {"id": "runValues"})
    spinDirection = parser.find("table", attrs = {"id": "spinAxis"})
    
    if not headerlist:
        pitchMovementHeaders = pitchMovement.findAll('th')
        pitchMovementHeadersList = [h.text.strip() for h in pitchMovementHeaders[:]]

        pitchTrackingHeaders = pitchTracking.findAll('th')
        pitchTrackingHeadersList = [h.text.strip() for h in pitchTrackingHeaders[:]]

        runValuesHeaders = runValues.findAll('th')
        runValuesHeadersList = [h.text.strip() for h in runValuesHeaders[:]]

        spinDirectionHeaders = spinDirection.findAll('th')
        spinDirectionHeadersList = [h.text.strip() for h in spinDirectionHeaders[:]]
    
        pitchMovementHeadersList_2 = pitchMovementHeadersList[2:]
        pitchMovementHeadersList_2[0] = 'Name'
        headerlist.extend(pitchMovementHeadersList_2)
        headerlist.extend(pitchTrackingHeadersList[3:])
        headerlist.extend(runValuesHeadersList[3:])
        headerlist.extend(spinDirectionHeadersList[3:])

    pitchMovementRows = pitchMovement.findAll('tr')[2:]
    pitchTrackingRows = pitchTracking.findAll('tr')[1:]
    runValuesRows = runValues.findAll('tr')[1:]
    spinDirectionRows = spinDirection.findAll('tr')[1:]
    
    pitchMovement_stats = []
    pitchTracking_stats = []
    runValues_stats = []
    spinDirection_stats = []
    
    for i in range(len(pitchMovementRows)):
        row_cells = []
        for td in pitchMovementRows[i].findAll('td'):
            row_cells.append(td.getText().strip())
        pitchMovement_stats.append(row_cells)

    for i in range(len(pitchTrackingRows)):
        row_cells = []
        for td in pitchTrackingRows[i].findAll('td'):
            row_cells.append(td.getText().strip())
        pitchTracking_stats.append(row_cells)

    for i in range(len(runValuesRows)):
        row_cells = []
        for td in runValuesRows[i].findAll('td'):
            row_cells.append(td.getText().strip())
        runValues_stats.append(row_cells)

    for i in range(len(spinDirectionRows)):
        row_cells = []
        for td in spinDirectionRows[i].findAll('td'):
            row_cells.append(td.getText().strip())
        spinDirection_stats.append(row_cells)

    if (len(pitchTrackingRows) > len(spinDirectionRows)):
        missing_stats(spinDirectionRows, pitchTrackingRows)
    elif (len(pitchTrackingRows) > len(runValuesRows)):
        missing_stats(runValuesRows, pitchTrackingRows)
    elif (len(pitchTrackingRows) > len(pitchMovementRows)):
        missing_stats(pitchMovementRows, pitchTrackingRows)   

    for i in range(len(pitchMovementRows)):
        for pitch in pitchMovement_stats:
            if pitch[1] == pitchTracking_stats[i][1] and pitch[0] == pitchTracking_stats[i][0]:
                pitch.extend(pitchTracking_stats[i][3:])

    for i in range(len(pitchMovementRows)):
        for pitch in pitchMovement_stats:
            if pitch[1] == runValues_stats[i][1] and pitch[0] == runValues_stats[i][0]:
                pitch.extend(runValues_stats[i][3:])

    for i in range(len(pitchMovementRows)):
        for pitch in pitchMovement_stats:
            if pitch[1] == spinDirection_stats[i][1] and pitch[0] == spinDirection_stats[i][0]:
                pitch.extend(spinDirection_stats[i][3:])

    for pitch in pitchMovement_stats: 
        pitch.insert(0, player[1])
        pitcher_stats.append(pitch)

driver.quit()
df = pd.DataFrame(pitchMovement_stats, columns=headerlist)

In [ ]:
pitchMovement_stats

In [ ]:
pitcher_stats.(pitchMovement_stats)

In [ ]:
pitcher_stats

In [ ]:
driver.quit()

In [ ]:
for i in range(len(pitchMovementRows)):
    for pitch in pitcher1_stats:
        if pitch[1] == pitcher2_stats[i][1]:
            pitch.extend(pitcher2_stats[i][3:])

In [ ]:
for i in range(len(pitchMovementRows)):
    for pitch in pitcher1_stats:
        if pitch[1] == pitcher3_stats[i][1]:
            pitch.extend(pitcher3_stats[i][3:])

In [ ]:
for i in range(len(pitchMovementRows)):
    for pitch in pitcher1_stats:
        if pitch[1] == pitcher4_stats[i][1]:
            pitch.extend(pitcher4_stats[i][3:])

In [ ]:
for pitch in pitcher1_stats[1:]: 
    pitch.insert(0, 'Brandon')

In [ ]:
df2 = pd.DataFrame(pitcher_stats, columns=headerlist)

In [ ]:
df2

In [ ]:
pitchMovementRows
# len(pitchTrackingRows)
# len(runValuesRows)
# len(spinDirectionRows)

In [ ]:
def missing_stats(pitches_less, pitches_more):  
    pitches_needed = []
    for row in pitches_more:
        yearPitch = row.findAll('td')[0].getText().strip() + row.findAll('td')[1].getText().strip()
        pitches_needed.append(yearPitch)
    
    pitches_got = []
    for row in pitches_less:
        yearPitch = row.findAll('td')[0].getText().strip() + row.findAll('td')[1].getText().strip()
        pitches_got.append(yearPitch)

    missing_pitch = []
    for pitch in pitches_needed:
        if pitch not in pitches_got:
            missing_pitch.append(pitch)

    missing_pitch_arr = []
    missing_pitch_arr.append(missing_pitch[0][:4])
    missing_pitch_arr.append(missing_pitch[0][4:])

    for stat in spinDirectionRows[2:]:
        missing_pitch_arr.append('') # appends empty string for value to keep the correct amount of cols
    
    spinDirection_stats.append(missing_pitch_arr) # appends missing pitch

In [ ]:
missing_stats(spinDirectionRows, pitchTrackingRows)

In [ ]:
spinDirection_stats

In [ ]:
if (len(pitchTrackingRows) > len(pitchMovementRows)) {
    
}

In [ ]:
spinDirection_stats = []
for i in range(len(spinDirectionRows)):
        row_cells = []
        for td in spinDirectionRows[i].findAll('td'):
            row_cells.append(td.getText().strip())
        spinDirection_stats.append(row_cells)
spinDirection_stats 

In [ ]:
pitches_needed = []
for row in pitchTrackingRows:
    yearPitch = row.findAll('td')[0].getText().strip() + row.findAll('td')[1].getText().strip()
    pitches_needed.append(yearPitch)
pitches_needed    

In [ ]:
pitches_got = []
for row in spinDirectionRows:
    yearPitch = row.findAll('td')[0].getText().strip() + row.findAll('td')[1].getText().strip()
    pitches_got.append(yearPitch)

missing_pitch = []
for pitch in pitches_needed:
    if pitch not in pitches_got:
        missing_pitch.append(pitch)


In [ ]:
missing_pitch_arr = []
for pitch in missing_pitch:
    missing_pitch_arr.append(pitch[0][:4])
    missing_pitch_arr.append([0][4:])   
    

In [ ]:
missing_pitch_arr

In [ ]:
missing_pitch_year = missing_pitch[0][:4]

In [ ]:
missing_pitch_arr = []

In [ ]:
missing_pitch_type = missing_pitch[0][4:]

In [ ]:
missing_pitch_arr.append(missing_pitch[0][:4])
missing_pitch_arr.append(missing_pitch[0][4:])

In [ ]:
missing_pitch_arr

In [ ]:
for stat in spinDirectionRows[2:]:
    missing_pitch_arr.append('') # appends empty string for value to keep the correct amount of cols

In [ ]:
spinDirection_stats.append(missing_pitch_arr) # appends missing pitch

In [ ]:
# len(pitchMovement_stats)
# len(pitchTracking_stats)
# len(runValues_stats)
# len(spinDirection_stats)

In [ ]:
def largest_pitch_count():
    pitch_count = []
    pitch_count.append(pitchMovementRows)
    pitch_count.append(pitchTrackingRows)
    pitch_count.append(runValuesRows)
    pitch_count.append(spinDirectionRows)
    return max(pitch_count, key=len)

if (len(largest_pitch_count()) > 5):    
    print(largest_pitch_count)

# pitchMovementRows
# len(pitchTrackingRows)
# len(runValuesRows)
# len(spinDirectionRows)

In [ ]:
len(largest_pitch_count())

In [ ]:
len(pitch_count[-1])

In [ ]:
runValues_stats[0]

In [1]:
# from players import players

import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


payload = {}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:122.0) Gecko/20100101 Firefox/122.0',
  'Accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.5',
  'Accept-Encoding': 'gzip, deflate, br',
  'X-Requested-With': 'XMLHttpRequest',
  'Connection': 'keep-alive',
  'Referer': 'https://baseballsavant.mlb.com/savant-player/alek-thomas-677950?stats=statcast-r-hitting-mlb',
  'Cookie': '__cflb=02DiuJFcgbX2zpcHdRurfpjkBTD5BPySo6gPFqPfkQ556; AMCV_A65F776A5245B01B0A490D44%40AdobeOrg=1099438348%7CMCIDTS%7C19755%7CMCMID%7C08223760795725670020171971273569729942%7CMCAAMLH-1707411436%7C7%7CMCAAMB-1707411436%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1706813836s%7CNONE%7CMCAID%7CNONE%7CMCSYNCSOP%7C411-19762%7CvVersion%7C2.1.0; s_cc=true; s_sq=%5B%5BB%5D%5D; AMCVS_A65F776A5245B01B0A490D44%40AdobeOrg=1; s_vi=[CS]v1|32DDE6B6150B13BF-40000A6BE1F97D36[CE]; s_ecid=MCMID%7C08223760795725670020171971273569729942; _dd_s=rum=0&expire=1706807948264',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'TE': 'trailers'
}

handedness =['R', 'L']
pitcher_headerlist = []
hitter_headerlist = []
player_stats = []
pitcher_stats = []
players = [['dbacks', 'Miguel Castro', 'miguel-castro-612434', 'Pitcher']]

# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

timeout = 10

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# pitcher function
def missing_stats(pitches_less, pitches_more, pitches_less_stats):  
    pitches_needed = []
    for row in pitches_more:
        yearPitch = row.findAll('td')[0].getText().strip() + row.findAll('td')[1].getText().strip()
        pitches_needed.append(yearPitch)
    
    pitches_got = []
    for row in pitches_less:
        yearPitch = row.findAll('td')[0].getText().strip() + row.findAll('td')[1].getText().strip()
        pitches_got.append(yearPitch)

    missing_pitch = []
    for pitch in pitches_needed:
        if pitch not in pitches_got:
            missing_pitch.append(pitch)

    missing_pitch_arr = []
    for pitch in missing_pitch:
        missing_pitch_arr.append(pitch[:4])
        missing_pitch_arr.append(pitch[4:])   

    for stat in pitches_less[2:]:
        missing_pitch_arr.append('') # appends empty string for value to keep the correct amount of cols
    
    for pitch in missing_pitch_arr:
        pitches_less_stats.append(pitch) # appends missing pitch

def largest_pitch_count():
    pitch_count = []
    pitch_count.append(pitchMovementRows)
    pitch_count.append(pitchTrackingRows)
    pitch_count.append(runValuesRows)
    pitch_count.append(spinDirectionRows)
    return max(pitch_count, key=len)


for player in players:
    if player[-1] != "Pitcher":    
        for hand in handedness:
            url = f"https://baseballsavant.mlb.com/player-services/statcast-pitches-breakdown?playerId={player[2].split('-')[-1]}&position=8&hand={hand}&pitchBreakdown=pitches&timeFrame=yearly&season=&pitchType=&count=&updatePitches=true"

            r = requests.get(url, headers=headers)

            parser = BeautifulSoup(r.content, "html.parser")
            table = parser.find(id="detailedPitches")

            headersTH = table.find_all('th')
            hitter_headerlist = [h.text.strip() for h in headersTH[:]]
            hitter_headerlist.insert(0, "Name")
            hitter_headerlist.insert(2, "Team")
            hitter_headerlist.insert(3, "Hand") 
    
            rows = table.findAll('tr')[1:]
    
            for i in range(len(rows)):
                row_cells = []
                for td in rows[i].findAll('td'):
                    row_cells.append(td.getText().strip())
                row_cells.insert(0, player[1])
                row_cells.insert(2, player[0])
                row_cells.insert(3, hand)
                player_stats.append(row_cells)
    else:

        
        url = f"https://baseballsavant.mlb.com/savant-player/{player[2]}?stats=statcast-r-pitching-mlb"
        driver.get(url)

        career_minors_locator = (By.XPATH, '//*[@id="top-stats"]/div[1]/table/tbody/tr[2]/td[1]/span')
        career_minors_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(career_minors_locator))
        career_minors_text = career_minors_element.text

        if (career_minors_text != 'Career Minors'):

            detailedPitches_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="detailedPitches"]/tbody/tr[1]/td[2]/span'))
            WebDriverWait(driver, timeout).until(detailedPitches_present)

            src = driver.page_source
            parser = BeautifulSoup(src, "lxml")

            pitchMovement = parser.find("table", attrs = {"id": "pitchMovement"})
            pitchTracking = parser.find("table", attrs = {"id": "detailedPitches"})
            runValues = parser.find("table", attrs = {"id": "runValues"})
            spinDirection = parser.find("table", attrs = {"id": "spinAxis"})
    
            if not pitcher_headerlist:
                pitchMovementHeaders = pitchMovement.findAll('th')
                pitchMovementHeadersList = [h.text.strip() for h in pitchMovementHeaders[:]]

                pitchTrackingHeaders = pitchTracking.findAll('th')
                pitchTrackingHeadersList = [h.text.strip() for h in pitchTrackingHeaders[:]]

                runValuesHeaders = runValues.findAll('th')
                runValuesHeadersList = [h.text.strip() for h in runValuesHeaders[:]]

                spinDirectionHeaders = spinDirection.findAll('th')
                spinDirectionHeadersList = [h.text.strip() for h in spinDirectionHeaders[:]]
    
                pitchMovementHeadersList_2 = pitchMovementHeadersList[2:]
                pitchMovementHeadersList_2[0] = 'Name'
                pitcher_headerlist.extend(pitchMovementHeadersList_2)
                pitcher_headerlist.extend(pitchTrackingHeadersList[3:])
                pitcher_headerlist.extend(runValuesHeadersList[3:])
                pitcher_headerlist.extend(spinDirectionHeadersList[3:])

            pitchMovementRows = pitchMovement.findAll('tr')[2:]
            pitchTrackingRows = pitchTracking.findAll('tr')[1:]
            runValuesRows = runValues.findAll('tr')[1:]
            spinDirectionRows = spinDirection.findAll('tr')[1:]
    
            pitchMovement_stats = []
            pitchTracking_stats = []
            runValues_stats = []
            spinDirection_stats = []

            for i in range(len(pitchMovementRows)):
                row_cells = []
                for td in pitchMovementRows[i].findAll('td'):
                    row_cells.append(td.getText().strip())
                pitchMovement_stats.append(row_cells)

            for i in range(len(pitchTrackingRows)):
                row_cells = []
                for td in pitchTrackingRows[i].findAll('td'):
                    row_cells.append(td.getText().strip())
                pitchTracking_stats.append(row_cells)

            for i in range(len(runValuesRows)):
                row_cells = []
                for td in runValuesRows[i].findAll('td'):
                    row_cells.append(td.getText().strip())
                runValues_stats.append(row_cells)

            for i in range(len(spinDirectionRows)):
                row_cells = []
                for td in spinDirectionRows[i].findAll('td'):
                    row_cells.append(td.getText().strip())
                spinDirection_stats.append(row_cells)

            if (len(largest_pitch_count()) > len(spinDirectionRows)):
                missing_stats(spinDirectionRows, largest_pitch_count(), spinDirection_stats)
            if (len(largest_pitch_count()) > len(pitchTrackingRows)):
                missing_stats(pitchTrackingRows, largest_pitch_count(), pitchTracking_stats)   
            if (len(largest_pitch_count()) > len(runValuesRows)):
                missing_stats(runValuesRows, largest_pitch_count(), runValues_stats)
            if (len(largest_pitch_count()) > len(pitchMovementRows)):
                missing_stats(pitchMovementRows, largest_pitch_count(), pitchMovement_stats) 
            

            for i in range(len(pitchMovement_stats)):
                for pitch in pitchMovement_stats:
                    if pitch[1] == pitchTracking_stats[i][1] and pitch[0] == pitchTracking_stats[i][0]:
                        pitch.extend(pitchTracking_stats[i][3:])

            for i in range(len(pitchMovement_stats)):
                for pitch in pitchMovement_stats:
                    if pitch[1] == runValues_stats[i][1] and pitch[0] == runValues_stats[i][0]:
                        pitch.extend(runValues_stats[i][3:])

            for i in range(len(pitchMovement_stats)):
                for pitch in pitchMovement_stats:
                    if pitch[1] == spinDirection_stats[i][1] and pitch[0] == spinDirection_stats[i][0]:
                        pitch.extend(spinDirection_stats[i][3:])
    
            for pitch in pitchMovement_stats: 
                pitch.insert(0, player[1])
                pitcher_stats.append(pitch)

driver.quit()

pitchers_df = pd.DataFrame(pitcher_stats, columns=pitcher_headerlist)
hitters_df = pd.DataFrame(player_stats, columns=hitter_headerlist)

print(pitchers_df)
print(hitters_df)

             Name  Year            Pitch Team Hand    #   MPH Inches of Drop  \
0   Miguel Castro  2023           Sinker   AZ    R  428  96.8           24.7   
1   Miguel Castro  2023           Slider   AZ    R  420  83.8           37.0   
2   Miguel Castro  2023         Changeup   AZ    R  160  91.4           30.7   
3   Miguel Castro  2022         Changeup  NYY    R   96  92.4           31.6   
4   Miguel Castro  2022           Sinker  NYY    R  189  97.9           23.3   
5   Miguel Castro  2022           Slider  NYY    R  227  85.8           35.2   
6   Miguel Castro  2021         Changeup  NYM    R  270  92.1           30.3   
7   Miguel Castro  2021           Slider  NYM    R  433  85.7           36.4   
8   Miguel Castro  2021           Sinker  NYM    R  525  98.0           21.7   
9   Miguel Castro  2020         Changeup  NYM    R   89  92.3           28.7   
10  Miguel Castro  2020           Slider  NYM    R  148  86.8           34.1   
11  Miguel Castro  2020           Sinker

In [ ]:
driver.quit()

In [ ]:
level = parser.find("div", attrs = {"id": "mlb_career"})

In [ ]:
career_minors_locator = (By.XPATH, '//*[@id="top-stats"]/div[1]/table/tbody/tr[2]/td[1]/span')

In [ ]:
career_minors_locator.element

In [ ]:
career_minors = driver.findElement(By.xpath('//*[@id="top-stats"]/div[1]/table/tbody/tr[2]/td[1]/span'))

In [ ]:
career_minors

In [ ]:
career_minors_locator = (By.XPATH, '//*[@id="top-stats"]/div[1]/table/tbody/tr[2]/td[1]/span')
career_minors_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(career_minors_locator))

In [ ]:
career_minors_text = career_minors_element.text

In [ ]:
career_minors_text